In [1]:
from GUI import GUI
from layout.obj.Compo import Compo
import layout.lib.pairing as pairing
import layout.lib.repetition_recognition as rep
import layout.lib.draw as draw
from sklearn.cluster import DBSCAN

import cv2
import pandas as pd
import math
import numpy as np
import os
from os.path import join as pjoin

In [2]:
# gui = GUI(img_file='E:\\Mulong\\Datasets\\gui\\rico\\combined\\all\\10727.jpg', output_dir='E:\\Mulong\\Result\\rico-layout\\result')

In [3]:
gui = GUI(img_file='data/input/11.jpg')

In [4]:
gui.element_detection()
# gui.visualize_element_detection()

In [6]:
gui.layout_recognition()
gui.visualize_layout_recognition()

15 1 t-1 1
[Layout Recognition Completed in 0.492 s] Input: data/input/11.jpg Output: data/output\layout\11.json


In [6]:
gui.visualize_element_detection()

In [18]:
gui.visualize_layout_recognition()

In [6]:
gui.cvt_compos_json_to_dataframe()

In [7]:
gui.compos_df.repetitive_group_recognition(show=True)

In [8]:
gui.compos_df.pair_groups()

In [6]:
b = gui.visualize_compos_df('group')

In [63]:
gui.compos_df.compos_dataframe = df

In [6]:
df = gui.compos_df.compos_dataframe

In [8]:
pair = df.groupby('group_pair').groups[1]
pair_all_compos = df.loc[pair]

In [9]:
paired_groups = pair_all_compos.groupby('group').groups

In [10]:
paired_groups

{'nt-0': Int64Index([3, 5, 7], dtype='int64'),
 'nt-1': Int64Index([4, 6, 8], dtype='int64'),
 't-0': Int64Index([12, 14, 16], dtype='int64'),
 't-1': Int64Index([13, 17], dtype='int64')}

In [13]:
def calc_compo_related_position_in_its_paired_item(compos):
    related_pos = None
    for i in range(len(compos)):
        compo = compos.iloc[i]
        # get the position of the item to which the compo belongs     
        item = pair_all_compos[pair_all_compos['list_item'] == compo['list_item']]
        item_pos = item['column_min'].min(), item['row_min'].min()    
        
        if related_pos is None:
            related_pos = compo['column_min'] - item_pos[0], compo['row_min'] - item_pos[1], compo['column_max'] - item_pos[0], compo['row_max'] - item_pos[1]
        else:
            related_pos = (min(related_pos[0], compo['column_min'] - item_pos[0]), min(related_pos[1], compo['row_min'] - item_pos[1]), 
                           max(related_pos[2], compo['column_max'] - item_pos[0]), max(related_pos[3], compo['row_max'] - item_pos[1]))
    return related_pos

In [57]:
def calc_intersected_area(bound1, bound2):
    '''
    bound: [column_min, row_min, column_max, row_max]
    '''
    col_min_s = max(bound1[0], bound2[0])
    row_min_s = max(bound1[1], bound2[1])
    col_max_s = min(bound1[2], bound2[2])
    row_max_s = min(bound1[3], bound2[3])
    w = np.maximum(0, col_max_s - col_min_s)
    h = np.maximum(0, row_max_s - row_min_s)
    inter = w * h
    return inter

In [61]:
def find_missed_compo_by_iou_with_potential_area(potential_area):
    unpaired_compos = df[df['group_pair'] == -1]
    for i in range(len(unpaired_compos)):
        up = unpaired_compos.iloc[i]
        inter = calc_intersected_area((up['column_min'], up['row_min'], up['column_max'], up['row_max']), potential_area)    
        if inter > 0 and inter/up['area'] > 0.5:
            return up['id']
    return None

In [62]:
max_group_compo_num = max([len(paired_groups[i]) for i in paired_groups])
for i in paired_groups:
    # Identify abnormal groups that have fewer compos that others do    
    if len(paired_groups[i]) < max_group_compo_num:
        # calculate the related position of the group compos in their paired item
        group_compos = df.loc[paired_groups[i]]  # compos in the abnormal group     
        compo_related_pos = calc_compo_related_position_in_its_paired_item(group_compos)  # (column_min, row_min, column_max, row_max)
        
        # identify the abnormal item and its position 
        abnormal_items = pair_all_compos[~pair_all_compos['list_item'].isin(list(group_compos['list_item']))]   
        abnormal_items_grp = abnormal_items.groupby('list_item').groups
        for j in abnormal_items_grp:
            abnormal_item = abnormal_items.loc[abnormal_items_grp[j]]
            abnormal_item_pos = abnormal_item['column_min'].min(), abnormal_item['row_min'].min()    
        
            # calculate the potential missed compo area based on the related compo position and the absolute item position
            potential_missed_compo_area = (compo_related_pos[0] + abnormal_item_pos[0], compo_related_pos[1] + abnormal_item_pos[1],
                                           compo_related_pos[2] + abnormal_item_pos[0], compo_related_pos[3] + abnormal_item_pos[1])
        
            # find the potential missed compo through iou with the potential_missed_compo_area
            missed_compo_id = find_missed_compo_by_iou_with_potential_area(potential_missed_compo_area)
            if missed_compo_id:
                df.loc[missed_compo_id, 'group'] = i
                df.loc[missed_compo_id, 'list_itm'] = j

In [53]:
related_pos

(170, 56, 442, 108)

In [54]:
potential_missed_compo_area

(188, 454, 460, 506)

In [55]:
abnormal_items_grp

{1: Int64Index([5, 6, 14], dtype='int64')}

In [56]:
board = gui.img_resized.copy()
cv2.rectangle(board, potential_missed_compo_area[:2], potential_missed_compo_area[2:], (0,255,0), 2)
cv2.imshow('board', board)
cv2.waitKey()
cv2.destroyAllWindows()

In [8]:
gui.cvt_compos_json_to_dataframe()
gui.compos_df.repetitive_group_recognition(show=True)

In [7]:
gui.compos_df.calc_gap_in_group()

In [9]:
df = gui.compos_df.compos_dataframe
g_nt1 = df[df['group'] == 'nt-1']
g_t3 = df[df['group'] == 't-3']

In [10]:
pairing.match_two_groups_by_angles_and_y_distance(g_nt1, g_t3)

[-1, -1] Corner Angles: [1, -2] Center Angles: [10, 3]


D:\Anaconda\lib\site-packages\pandas\core\indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
D:\Anaconda\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


True